<h1>Database</h1>

In [71]:
# Libraries to install in the case of need

#!pip install sqlalchemy
#!pip install pymysql
#!pip install sqlalchemy-utils

In [4]:
import pandas as pd
import os
from sqlalchemy import create_engine,ForeignKey, MetaData, Table, Column, Integer, String,DECIMAL,TIMESTAMP,Insert,DateTime
from sqlalchemy_utils import create_database, database_exists
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session

<h4>Connect and Create Database<h4>

In [3]:
# Params to build Connection String
user = 'root'
password = 'admin'
host = 'localhost'
port = 3306
database = 'DigitalCurrency'
# Connection String
url = "mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
			user, password, host, port,database
		)
# CONNECT TO THE MYSQL DATABASE 
engine = create_engine(
		url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
			user, password, host, port,database
		))
# Create DB if not exists
if not database_exists(url):
    create_database(url)

In [4]:
# Define Tables by declarative base method
meta = MetaData()
Base = declarative_base()

class Currency(Base):
   __tablename__ = 'Currency'

   CurrencyId = Column(Integer, primary_key = True, autoincrement = True, nullable = False)
   Name = Column(String(250),nullable=False)
   Symbol = Column(String(250))
   Rank = Column(Integer)
   MainLink = Column(String(250))
   HistoricalLink = Column(String(250))
   GithubLink = Column(String(250))

class Tag(Base):
   __tablename__ = 'Tag'

   TagId = Column(Integer, primary_key = True, autoincrement = True, nullable = False)
   Name = Column(String(250),nullable=False)
   

class CurrencyTag(Base):
   __tablename__ = 'CurrencyTag'

   CurrencyTagId = Column(Integer, primary_key = True, autoincrement = True, nullable = False)
   TagId = Column(Integer,ForeignKey('Tag.TagId'),nullable=False)
   CurrencyId = Column(Integer,ForeignKey('Currency.CurrencyId'),nullable=False)


class CurrencyData(Base):
   __tablename__ = 'CurrencyData'

   CurrencyDataId = Column(Integer, primary_key = True, autoincrement = True, nullable = False)
   CurrencyId = Column(Integer,ForeignKey('Currency.CurrencyId'),nullable=False)
   TimeOpen = Column(DateTime)
   TimeClose = Column(DateTime)
   TimeHigh = Column(DateTime)
   TimeLow = Column(DateTime)
   Open = Column(DECIMAL(20,14))
   High = Column(DECIMAL(20,14))
   Low = Column(DECIMAL(20,14))
   Close = Column(DECIMAL(20,14))
   Volume = Column(DECIMAL(18,4))
   MarketCap = Column(DECIMAL(18,4))



class Language(Base):
   __tablename__ = 'Language'

   LanguageId = Column(Integer, primary_key = True, autoincrement = True, nullable = False)
   Name = Column(String(250),nullable=False)
   

class CurrencyLanguage(Base):
   __tablename__ = 'CurrencyLanguage'

   CurrencyLanguageId = Column(Integer, primary_key = True, autoincrement = True, nullable = False)
   LanguageId = Column(Integer,ForeignKey('Language.LanguageId'),nullable=False)
   CurrencyId = Column(Integer,ForeignKey('Currency.CurrencyId'),nullable=False)   


class CurrencyContributor(Base):
   __tablename__ = 'CurrencyContributor'

   CurrencyContributorId = Column(Integer, primary_key = True, autoincrement = True, nullable = False)
   CurrencyId = Column(Integer,ForeignKey('Currency.CurrencyId'),nullable=False)   
   Contributor = Column(String(250))

# Create Tables
Base.metadata.create_all(engine)
   

C:\Users\Mohsen\AppData\Local\Temp\ipykernel_20416\3775690541.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


<h4>Read and Insert Data<h4>

In [30]:
# Get and build url to read data from Scraping Section
base_url = os.getcwd().split('DatabaseSection')[0]
data_base_url = base_url + 'ScrapingSection\\'

In [31]:
# Read currencies & fill nan values
currencies = pd.read_csv(data_base_url + 'ScrapedData\\FirstTable.csv')
currencies['GitHubLink'].fillna('',inplace=True)

In [7]:
# Insert Currencies To DB
session = Session(engine)
for i in range(len(currencies)):
  session.add(Currency(Name=currencies.loc[i][1], Symbol=currencies.loc[i][2],
            Rank=(currencies.loc[i][0] + 1), MainLink=currencies.loc[i][3],
              HistoricalLink=currencies.loc[i][4], GithubLink=currencies.loc[i][5]))
session.commit()

In [8]:
# build file's prefix and sufix url that we'll use to read currencies's data
prefix_url = data_base_url + 'ScrapedData\\HistoricalData\\'
suffix_url = '_8_25_2022-8_25_2023_historical_data_coinmarketcap.csv'

# Insert Currencies's Data To DB
session = Session(engine)
for j in range(len(currencies)):
    curr_name = currencies.loc[j][1]
    data = pd.read_csv(prefix_url + curr_name + suffix_url,sep=';')
    # Get CurrencyId from DB
    curr_id = session.query(Currency).filter(Currency.Name == curr_name).first().CurrencyId
    for i in range(len(data)):
        session.add(CurrencyData(CurrencyId = curr_id
                                , TimeOpen = pd.to_datetime(data.loc[i][0].replace('T',' ').split('.')[0])
                                , TimeClose = pd.to_datetime(data.loc[i][1].replace('T',' ').split('.')[0])
                                , TimeHigh = pd.to_datetime(data.loc[i][2].replace('T',' ').split('.')[0])
                                , TimeLow = pd.to_datetime(data.loc[i][3].replace('T',' ').split('.')[0])
                                , Open = data.loc[i][4]
                                , High = data.loc[i][5]
                                , Low = data.loc[i][6]
                                , Close = data.loc[i][7]
                                , Volume = data.loc[i][8]
                                , MarketCap = data.loc[i][9]))
session.commit()

In [32]:
# Read Tags
tags = pd.read_csv(data_base_url + 'ScrapedData\\Tags.csv')

In [ ]:
# Get unique tags for tag table
tag_names = tags['Tag'].unique()
# Insert tags to DB
session = Session(engine)
for i in range(len(tag_names)):
    session.add(Tag(Name=tag_names[i]))
session.commit()

In [10]:
# Insert Currency Tags to DB
session = Session(engine)

for i in range(len(tags)):
    tag_name = tags.loc[i]['Tag']
    # Get TagId from DB
    tag_id = session.query(Tag).filter(Tag.Name == tag_name).first().TagId
    session.add(CurrencyTag(TagId = tag_id,CurrencyId = tags.loc[i]['CurrencyId']))
    
session.commit()

In [5]:
# Read CurrencyLanguages Data from csv
languages_data = pd.read_csv(data_base_url + 'ScrapedData\\CurrencyLanguages.csv')

In [236]:
# Get Unique Languages
unique_languages = languages_data['Language'].unique()
# Insert Languages To DB
session = Session(engine)
for i in range(len(unique_languages)):
    session.add(Language(Name=unique_languages[i]))
session.commit()

In [6]:
# Insert Currency Languages To DB
session = Session(engine)
for i in range(len(languages_data)):
    # Get LanguageId from DB
    lang_name = languages_data.loc[i]['Language']
    lang_id = session.query(Language).filter(Language.Name == lang_name).first().LanguageId
    session.add(CurrencyLanguage(LanguageId = lang_id, CurrencyId = languages_data.loc[i]['CurrencyId']))
session.commit()

In [7]:
# Read CurrencyContributors Data from csv
contributors_data = pd.read_csv(data_base_url + 'ScrapedData\\CurrencyContributors.csv')

In [8]:
# Insert Currency Contributors To DB
session = Session(engine)
for i in range(len(contributors_data)):
    session.add(CurrencyContributor(CurrencyId = contributors_data.loc[i]['CurrencyId']
                                    ,Contributor = contributors_data.loc[i]['Contributor']))
session.commit()